In [1]:
import pandas as pd

movies_df = pd.read_csv("..\\data\\ml - 1m\\movies.csv")
ratings_df = pd.read_csv("..\\data\\ml - 1m\\ratings.csv")  
imdb_df = pd.read_csv("..\\data\\movie_metadata.csv")
tags_df = pd.read_csv("..\\data\\ml - 1m\\tags.csv")
links_df = pd.read_csv("..\\data\\ml - 1m\\links.csv")

In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, Float, Text, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship

# Create persistent database file
engine = create_engine('sqlite:///../data/movies.db', echo=False)
Base = declarative_base()

class User(Base):
    __tablename__ = 'users'
    user_id = Column(Integer, primary_key=True)
    
    ratings = relationship("Rating", back_populates="user")

class Movie(Base):
    __tablename__ = 'movies'
    movie_id = Column(Integer, primary_key=True)
    title = Column(String)
    genres = Column(String)  # Pipe-separated from MovieLens
    imdb_id = Column(Integer)  # From links.csv
    imdb_url = Column(String)  # From IMDb metadata
    plot_keywords = Column(Text)  # From IMDb metadata
    
    ratings = relationship("Rating", back_populates="movie")

class Rating(Base):
    __tablename__ = 'ratings'
    user_id = Column(Integer, ForeignKey('users.user_id'), primary_key=True)
    movie_id = Column(Integer, ForeignKey('movies.movie_id'), primary_key=True)
    rating = Column(Float)
    timestamp = Column(Integer)
    
    user = relationship("User", back_populates="ratings")
    movie = relationship("Movie", back_populates="ratings")

# Create tables
Base.metadata.create_all(engine)
print("Database tables created successfully!")

Database tables created successfully!


C:\Users\Beka\AppData\Local\Temp\ipykernel_13476\1395398340.py:7: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [3]:
import re

def extract_imdb_id_from_link(link):
    """Convert 'http://.../tt0114709/' → 114709 (integer)"""
    match = re.search(r'tt(\d+)', link)
    return int(match.group(1)) if match else None

In [4]:
imdb_df["imdbId"] = imdb_df["movie_imdb_link"].apply(extract_imdb_id_from_link)

print("IMDb IDs extracted:")
print(imdb_df[['movie_title', 'movie_imdb_link', 'imdbId']].head(5))

IMDb IDs extracted:
                                         movie_title  \
0                                            Avatar    
1          Pirates of the Caribbean: At World's End    
2                                           Spectre    
3                             The Dark Knight Rises    
4  Star Wars: Episode VII - The Force Awakens    ...   

                                     movie_imdb_link   imdbId  
0  http://www.imdb.com/title/tt0499549/?ref_=fn_t...   499549  
1  http://www.imdb.com/title/tt0449088/?ref_=fn_t...   449088  
2  http://www.imdb.com/title/tt2379713/?ref_=fn_t...  2379713  
3  http://www.imdb.com/title/tt1345836/?ref_=fn_t...  1345836  
4  http://www.imdb.com/title/tt5289954/?ref_=fn_t...  5289954  


In [5]:
movies_with_links = movies_df.merge(links_df, on='movieId', how='inner')

print("Movies with links shape:" , movies_with_links.shape)
print(movies_with_links.head())

Movies with links shape: (9742, 5)
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  imdbId   tmdbId  
0  Adventure|Animation|Children|Comedy|Fantasy  114709    862.0  
1                   Adventure|Children|Fantasy  113497   8844.0  
2                               Comedy|Romance  113228  15602.0  
3                         Comedy|Drama|Romance  114885  31357.0  
4                                       Comedy  113041  11862.0  


In [ ]:
# Merge with IMDb metadata
movies_enriched = movies_with_links.merge(
    imdb_df[['imdbId', 'plot_keywords', 'movie_imdb_link']], 
    on='imdbId', 
    how='left'
)

print("Final enriched movies shape:", movies_enriched.shape)
print("\nSample enriched movie:")
print(movies_enriched[['movieId', 'title', 'genres', 'imdbId', 'plot_keywords']].head(3))

Final enriched movies shape: (9852, 7)

Sample enriched movie:
   movieId                    title  \
0        1         Toy Story (1995)   
1        2           Jumanji (1995)   
2        3  Grumpier Old Men (1995)   

                                        genres  imdbId  \
0  Adventure|Animation|Children|Comedy|Fantasy  114709   
1                   Adventure|Children|Fantasy  113497   
2                               Comedy|Romance  113228   

                            plot_keywords  
0  claw crane|cowboy|jealousy|rivalry|toy  
1                                     NaN  
2                                     NaN  


In [7]:
# Remove duplicates: keep first match per movieId
movies_clean = movies_enriched.drop_duplicates(subset='movieId', keep='first')

print("After deduplication:")
print(f"  Original: {len(movies_enriched)} rows")
print(f"  Cleaned:  {len(movies_clean)} rows")
print(f"  Movies with plot_keywords: {movies_clean['plot_keywords'].notna().sum()} / {len(movies_clean)}")

# Preview cleanup result
print("\nSample after cleanup:")
print(movies_clean[['movieId', 'title', 'genres', 'imdbId', 'plot_keywords']].head(3))

After deduplication:
  Original: 9852 rows
  Cleaned:  9742 rows
  Movies with plot_keywords: 3530 / 9742

Sample after cleanup:
   movieId                    title  \
0        1         Toy Story (1995)   
1        2           Jumanji (1995)   
2        3  Grumpier Old Men (1995)   

                                        genres  imdbId  \
0  Adventure|Animation|Children|Comedy|Fantasy  114709   
1                   Adventure|Children|Fantasy  113497   
2                               Comedy|Romance  113228   

                            plot_keywords  
0  claw crane|cowboy|jealousy|rivalry|toy  
1                                     NaN  
2                                     NaN  


In [8]:
# Prepare movies data for insertion
movies_to_insert = movies_clean.rename(columns={
    'movieId': 'movie_id',
    'imdbId': 'imdb_id',
    'movie_imdb_link': 'imdb_url',
    'plot_keywords': 'plot_keywords'
})

# Select only columns that match our table schema
movies_final = movies_to_insert[[
    'movie_id', 'title', 'genres', 'imdb_id', 'imdb_url', 'plot_keywords'
]]

# Insert into database
movies_final.to_sql('movies', con=engine, if_exists='append', index=False)

print(f"✓ Inserted {len(movies_final)} movies into database")
print(f"  - With plot keywords: {movies_final['plot_keywords'].notna().sum()}")

✓ Inserted 9742 movies into database
  - With plot keywords: 3530


In [9]:
# Extract unique users from ratings data
unique_users = ratings_df[['userId']].drop_duplicates()
unique_users = unique_users.rename(columns={'userId': 'user_id'})

# Insert users
unique_users.to_sql('users', con=engine, if_exists='append', index=False)

print(f"✓ Inserted {len(unique_users)} unique users into database")

✓ Inserted 610 unique users into database


In [ ]:
# inserting with error handling
try:
    ratings_to_insert = ratings_df.rename(columns={
        'userId': 'user_id',
        'movieId': 'movie_id'
    })
    
    ratings_to_insert.to_sql('ratings', con=engine, if_exists='append', index=False)
    print(f"✓ Successfully inserted {len(ratings_to_insert)} ratings")
    
except Exception as e:
    print(f"❌ Error during insertion: {e}")
    print("\nChecking data types:")
    print(ratings_to_insert.dtypes)

✓ Successfully inserted 100836 ratings


In [ ]:
# Quick verification queries
print("Database verification:")
print(f"  Movies:    {pd.read_sql('SELECT COUNT(*) as c FROM movies', engine).iloc[0]['c']}")
print(f"  Users:     {pd.read_sql('SELECT COUNT(*) as c FROM users', engine).iloc[0]['c']}")
print(f"  Ratings:   {pd.read_sql('SELECT COUNT(*) as c FROM ratings', engine).iloc[0]['c']}")

print("\nSample movies:")
sample_movies = pd.read_sql('SELECT * FROM movies LIMIT 3', engine)
print(sample_movies[['movie_id', 'title', 'genres', 'imdb_id']].to_string(index=False))

Database verification:
  Movies:    9742
  Users:     610
  Ratings:   100836

Sample movies:
 movie_id                   title                                      genres  imdb_id
        1        Toy Story (1995) Adventure|Animation|Children|Comedy|Fantasy   114709
        2          Jumanji (1995)                  Adventure|Children|Fantasy   113497
        3 Grumpier Old Men (1995)                              Comedy|Romance   113228
